# Python Field Control Model
Here is an implementation I quickly put together of the paper "*Wide Open Spaces: A statistical technique for measuring space creation in professional soccer*" for NFL data. This has not yet been completely error checked, but should work on most of the data, barring any outliers. I have made a few changes to the model which I will go over in more detail later, but the core is exactly the same. I will cover how this model is built mathematically in the near future, so that you can create changes to it to better reflect American Football rather than soccer.

Markdown of this notebook and further explanations will be added over the next week

# Setup
Before we get into the model, let's first load in all of the necessary libraries, and the dataset that we will be using.

## Library Imports

In [2]:
# Magic
# %matplotlib inline

# Utility Libraries
from datetime import datetime
import pytz

# HTML 
from IPython.display import HTML

# Computation Libraries
import numpy as np
import pandas as pd
import scipy.stats as stats

# Plotting libraries
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from matplotlib.patches import Rectangle, Arrow

# Modelling Libraries
import tensorflow as tf

## Dataset imports
Usual dataset imports using *read_csv()*. I am only loading in week 1 data for this notebook as it is an example

In [3]:
path_shared = './data/{}'

games_df = pd.read_csv(path_shared.format('games.csv'))
plays_df = pd.read_csv(path_shared.format('plays.csv'))
players_df = pd.read_csv(path_shared.format('players.csv'))
week_df = pd.read_csv(path_shared.format('week1_norm.csv'))

I will perform the field control model for a single play. Let's select all of the positional data for a single play in a single game. You can change **game_idx** and **play_idx** to see whatever play you wish. Let's use *df.head()* to show the first few rows of the dataset that we will be making a control model from.

There are a few things to note here. All of the positional units are in yards. The velocity is in $\frac{yards}{s}$, and the acceleration is in $\frac{yards}{s^2}$. At each time point, we will have a row for every entity of interest in that particular play. Note that all 22 players on the field are not shown, only the players that are relevant to the pass play. **o** stands for the orientation of the player (i.e. which way their trunk is facing) and **dir** is the movement direction of the player. These two values will rarely be the same, so it is important to keep track of both of them during our model. Also note that these angles are in degrees, calculated from due north, clockwise. Generally, angles will be measured from due east counter-clockwise, so we must be careful to either remap these angles, or change model formulation to ensure we are doing the correct thing.

Also, not that the width of the pitch is 120 by 53.3. This will be useful to plot the pitch in the correct aspect ratio.

In [4]:
# Select the game and play that you wish to see in week 1

game_idx = 5
play_idx = -15

unique_game_ids = week_df.gameId.unique()
unique_play_ids = week_df[week_df.gameId == unique_game_ids[game_idx]].playId.unique()

play_df = week_df[week_df.playId == unique_play_ids[play_idx]].sort_values(by = 'time')
play_df.head()

,time,x,y,s,a,dis,o,dir,event,nflId,...,frameId,team,gameId,playId,playDirection,route,los,los_norm,x_norm,y_norm
492218,2018-09-09T19:46:59.400Z,32.55,29.36,0.00,0.00,0.00,312.34,272.46,None,2495143.0,...,1,away,2018090902,3812,left,NaN,31.1,40,38.55,23.94
492231,2018-09-09T19:46:59.400Z,31.10,29.32,0.00,0.00,0.00,NaN,NaN,None,NaN,...,1,football,2018090902,3812,left,NaN,31.1,40,40.00,23.98
492230,2018-09-09T19:46:59.400Z,26.74,34.65,0.10,0.11,0.01,82.85,75.34,None,2560789.0,...,1,home,2018090902,3812,left,NaN,31.1,40,44.36,18.65
492229,2018-09-09T19:46:59.400Z,27.90,17.27,0.25,0.71,0.02,74.99,198.19,None,2558976.0,...,1,home,2018090902,3812,left,NaN,31.1,40,43.20,36.03
492228,2018-09-09T19:46:59.400Z,25.97,29.60,0.07,0.18,0.01,76.49,288.99,None,2558851.0,...,1,home,2018090902,3812,left,NaN,31.1,40,45.13,23.70


# Animation
## Animation Class

First let us make an animation that will show all of the entities moving across the pitch over time through the play. We are in luck that we have timestamps for each of the timepoints when the measurement was taken. This allows us to make a very precise estimate as to what the sampling rate is, and thus we can set an accurate framerate for the animation to simulate the play at "real time". This is calculated on line 26 as `self._mean_interval_ms`. 

In this animation we plot:
- Player positions
- Line of scrimmage
- Ball
- Velocity vectors of the players
- Orientation of players
- Player last name
- Player position
- Player number

On small screens this may be too much information, but this just shows how you can do it. Feel free to remove as you wish. I will not go into this animation class in too much detail as it is not the focus of this notebook. Feel free to go through it, it is mostly just using matplotlib and basic math to show what we want.

In [ ]:
class AnimatePlay:
    def __init__(self, play_df, plot_size_len) -> None:
        """Initializes the datasets used to animate the play.

        Parameters
        ----------
        play_df : DataFrame
            Dataframe corresponding to the play information for the play that requires
            animation. This data will come from the weeks dataframe and contains position
            and velocity information for each of the players and the football.

        Returns
        -------
        None
        """
        self._MAX_FIELD_Y = 53.3
        self._MAX_FIELD_X = 120
        self._MAX_FIELD_PLAYERS = 22
        
        self._CPLT = sns.color_palette("husl", 2)
        self._frame_data = play_df
        self._times = sorted(play_df.time.unique())
        self._stream = self.data_stream()
        
        self._date_format = "%Y-%m-%dT%H:%M:%S.%fZ" 
        self._mean_interval_ms = np.mean([delta.microseconds/1000 for delta in np.diff(np.array([pytz.timezone('US/Eastern').localize(datetime.strptime(date_string, self._date_format)) for date_string in self._times]))])
        
        self._fig = plt.figure(figsize = (plot_size_len, plot_size_len*(self._MAX_FIELD_Y/self._MAX_FIELD_X)))

        self._ax_field = plt.gca()
        
        self._ax_home = self._ax_field.twinx()
        self._ax_away = self._ax_field.twinx()
        self._ax_jersey = self._ax_field.twinx()

        self.ani = animation.FuncAnimation(self._fig, self.update, frames=len(self._times), interval = self._mean_interval_ms, 
                                          init_func=self.setup_plot, blit=False)
        
        plt.close()
       
    @staticmethod
    def set_axis_plots(ax, max_x, max_y) -> None:
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)

        ax.set_xlim([0, max_x])
        ax.set_ylim([0, max_y])
        
    @staticmethod
    def convert_orientation(x):
        return (-x + 90)%360
    
    @staticmethod
    def polar_to_z(r, theta):
        return r * np.exp( 1j * theta)
    
    @staticmethod
    def deg_to_rad(deg):
        return deg*np.pi/180
        
    def data_stream(self):
        for time in self._times:
            yield self._frame_data[self._frame_data.time == time]
    
    def setup_plot(self): 
        self.set_axis_plots(self._ax_field, self._MAX_FIELD_X, self._MAX_FIELD_Y)
        
        ball_snap_df = self._frame_data[(self._frame_data.event == 'ball_snap') & (self._frame_data.team == 'football')]
        self._ax_field.axvline(ball_snap_df.x.to_numpy()[0], color = 'k', linestyle = '--')
        
        self.set_axis_plots(self._ax_home, self._MAX_FIELD_X, self._MAX_FIELD_Y)
        self.set_axis_plots(self._ax_away, self._MAX_FIELD_X, self._MAX_FIELD_Y)
        self.set_axis_plots(self._ax_jersey, self._MAX_FIELD_X, self._MAX_FIELD_Y)
        
        for idx in range(10,120,10):
            self._ax_field.axvline(idx, color = 'k', linestyle = '-', alpha = 0.05)
            
        self._scat_field = self._ax_field.scatter([], [], s = 100, color = 'black')
        self._scat_home = self._ax_home.scatter([], [], s = 500, color = self._CPLT[0], edgecolors = 'k')
        self._scat_away = self._ax_away.scatter([], [], s = 500, color = self._CPLT[1], edgecolors = 'k')
        
        self._scat_jersey_list = []
        self._scat_number_list = []
        self._scat_name_list = []
        self._a_dir_list = []
        self._a_or_list = []
        for _ in range(self._MAX_FIELD_PLAYERS):
            self._scat_jersey_list.append(self._ax_jersey.text(0, 0, '', horizontalalignment = 'center', verticalalignment = 'center', c = 'white'))
            self._scat_number_list.append(self._ax_jersey.text(0, 0, '', horizontalalignment = 'center', verticalalignment = 'center', c = 'black'))
            self._scat_name_list.append(self._ax_jersey.text(0, 0, '', horizontalalignment = 'center', verticalalignment = 'center', c = 'black'))
            
            self._a_dir_list.append(self._ax_field.add_patch(Arrow(0, 0, 0, 0, color = 'k')))
            self._a_or_list.append(self._ax_field.add_patch(Arrow(0, 0, 0, 0, color = 'k')))
            
        return (self._scat_field, self._scat_home, self._scat_away, *self._scat_jersey_list, *self._scat_number_list, *self._scat_name_list)
        
    def update(self, anim_frame):
        pos_df = next(self._stream)
        
        for label in pos_df.team.unique():
            label_data = pos_df[pos_df.team == label]

            if label == 'football':
                self._scat_field.set_offsets(np.hstack([label_data.x, label_data.y]))
            elif label == 'home':
                self._scat_home.set_offsets(np.vstack([label_data.x, label_data.y]).T)
            elif label == 'away':
                self._scat_away.set_offsets(np.vstack([label_data.x, label_data.y]).T)

        jersey_df = pos_df[pos_df.jerseyNumber.notnull()]
        
        for (index, row) in pos_df[pos_df.jerseyNumber.notnull()].reset_index().iterrows():
            self._scat_jersey_list[index].set_position((row.x, row.y))
            self._scat_jersey_list[index].set_text(row.position)
            self._scat_number_list[index].set_position((row.x, row.y+1.9))
            self._scat_number_list[index].set_text(int(row.jerseyNumber))
            self._scat_name_list[index].set_position((row.x, row.y-1.9))
            self._scat_name_list[index].set_text(row.displayName.split()[-1])
            
            player_orientation_rad = self.deg_to_rad(self.convert_orientation(row.o))
            player_direction_rad = self.deg_to_rad(self.convert_orientation(row.dir))
            player_speed = row.s
            
            player_vel = np.array([np.real(self.polar_to_z(player_speed, player_direction_rad)), np.imag(self.polar_to_z(player_speed, player_direction_rad))])
            player_orient = np.array([np.real(self.polar_to_z(2, player_orientation_rad)), np.imag(self.polar_to_z(2, player_orientation_rad))])
            
            self._a_dir_list[index].remove()
            self._a_dir_list[index] = self._ax_field.add_patch(Arrow(row.x, row.y, player_vel[0], player_vel[1], color = 'k'))
            
            self._a_or_list[index].remove()
            self._a_or_list[index] = self._ax_field.add_patch(Arrow(row.x, row.y, player_orient[0], player_orient[1], color = 'grey', width = 2))
                
        
        return (self._scat_field, self._scat_home, self._scat_away, *self._scat_jersey_list, *self._scat_number_list, *self._scat_name_list)

## Animation Result
We can now use this class to create an animation of the entire play. Just instantiate the class, then convert to JS.

In [ ]:
animated_play = AnimatePlay(play_df, 20)
HTML(animated_play.ani.to_jshtml())

# Pitch Control Model
## Background
The pitch control model that we will be implementing consists of two main components.

1. Individual player influence
2. Team Pitch Control

Team pitch control falls into place relatively easily once we can model individual player influence, so most of our efforts will be concentrated on point 1.

## Distance From Ball
Let's think about how much influence any one player can have given their individual distance from the ball. If we imagine two players standing still and facing each other, 5 yards apart, and one throws the football, how much can the other possible move and still catch the football? Without accounting for other variables that may influence this like individual player acceleration, ball speed, etc., we can say maybe 2 yards on either side, and maybe 1 yard forward and backward. This will give us a 'region of influence' for the receiving player. In a very simple case, we can model a region like this as a gaussian distribution, with the mean centered at the player, and a covariance describing this region. This will be highly subjective based on modelling assumptions, but we can model something as follows. Note that when we create the covariance matrix, I am assuming no covariance. The variance occurs only on the individual axes, meaning the covariance matrix is diagonal.

Red refers to the receiver, and black refers to the thrower. We are currently only drawing the influence of the receiver.

In [ ]:
# Player positions
player_throw = np.array([0, 0])
player_catch = np.array([5, 0])

# Setup required parameters of multivariate normal
sigma = np.array([[1**2, 0], [0, 2**2]])
mu = player_catch

# Setup Random Variable with underlying distribution of multivariate normal
rv = stats.multivariate_normal(mu, sigma)

# Create grid of points to pass into multivariate normal
x, y = np.mgrid[-5:10:.01, -5:5:.01]
pos = np.dstack((x, y))

# Plot players and 
fig = plt.figure(figsize = (10, 10))
ax = plt.gca()
ax_influence = ax.twinx()

# Plot the figure
ax_influence.contourf(x, y, rv.pdf(pos), alpha = 0.3, cmap = 'Reds')

ax.scatter(player_throw[0], player_throw[1], s = 300, color = 'k', label = 'thrower')
ax.scatter(player_catch[0], player_catch[1], s = 300, color = 'r', label = 'receiver')

ax.set_xlim([-5, 10])
ax.set_ylim([-5, 5])

ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)
ax_influence.yaxis.set_visible(False)

ax.set

fig.show()

How about the person throwing the ball? How much influence will they have at the moment they throw the ball? Again this would be dependent on many different factors, but let's keep it simple for now. At this moment in time, they could probably influence about 1 yard in any direction. Note that we are not looking at the influence gained from throwing the ball, but rather at the physical influence of any player as a function of distance from the ball. If we think of black as a wide receiver who has just caught the ball, all he can really do is run. We are assuming he has some influence on the space 1 yard in any direction around him. Let's plot this on our graph as well

In [ ]:
# Player positions
player_throw = np.array([0, 0])
player_catch = np.array([5, 0])

# Setup required parameters of multivariate normal
sigma = np.array([[1**2, 0], [0, 2**2]])
mu = player_catch

sigma_throw = np.array([[1**2, 0], [0, 1**2]])
mu_throw = player_throw

# Setup Random Variable with underlying distribution of multivariate normal
rv = stats.multivariate_normal(mu, sigma)
rv_throw = stats.multivariate_normal(mu_throw, sigma_throw)

# Create grid of points to pass into multivariate normal
x, y = np.mgrid[-5:10:.01, -5:5:.01]
pos = np.dstack((x, y))

# Plot players and 
fig = plt.figure(figsize = (10, 10))
ax = plt.gca()
ax_influence = ax.twinx()

# Plot the figure
ax_influence.contourf(x, y, rv.pdf(pos), alpha = 0.3, cmap = 'Reds')
ax_influence.contourf(x, y, rv_throw.pdf(pos), alpha = 0.3, cmap = 'Greys')


ax.scatter(player_throw[0], player_throw[1], s = 300, color = 'k', label = 'thrower')
ax.scatter(player_catch[0], player_catch[1], s = 300, color = 'r', label = 'receiver')

ax.set_xlim([-5, 10])
ax.set_ylim([-5, 5])

ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)
ax_influence.yaxis.set_visible(False)

ax.set

fig.show()

So now we can see the phenomenon that we want to model. The further away you are from the ball, the wider your region of influence will be, but it will also be weaker. This is because if you are far away from the ball, it will take time for the ball to reach you. In this time, you can move further. Influence is a very abstract concept, so feel free to model with any assumptions you like, but we will model this model using this assumption. The further you are from the ball, the more influence you will have, up to a certain distance. For soccer this makes a lot of sense as the game is continuous, however for American football, we may have to change this. Once a receiver catches the ball, another receiver on the other end of the field will have no influence on the play. We have not done this yet, but this is a weakness of the current model.

### Distance from the ball influence function
The function for distance from the ball vs influence is set relatively arbitrarily for now. I have done something similar to the function that is provided in the paper, but this would need to be tuned using subject matter experts from the NFL. The current function we will use is

$f(x) = 4 + (\frac{6}{18^2})(x^2) \textrm{   if   } x < 18$

$f(x) = 10 \textrm{   otherwise   }$

Note that after 18 yards, the radius of influence is constant.

In [ ]:
x = np.arange(0, 30, 0.1)

@np.vectorize
def radius_influence(x):
    assert x >= 0

    if x <= 18:
        return 4 + (6/(18**2))*(x**2)
    else:
        return 10
    
plt.plot(x, radius_influence(x), 'k')
plt.xlabel('Distance From Ball')
plt.ylabel('Influence Radius')
plt.title('Influence Radius as a function of distance from ball')

plt.show()

## Modelling how velocity affects region of influence
As a player is running in a high velocity in a specific direction, their region of influence should change to reflect the fact that it would be difficult for them to influence anything that is not in line with or nearly in line with the direction they are running. However they can also influence a much larger region in the direction that they are running because they are already running at a high velocity. This means we need to create a region of influence that is elongated in the region that the player is running, as a function of speed. When they are stationary, the variances in either direction should be the same, but as they start running, the difference between the variances should increase. For now, we will just focus on what occurs when the player runs on the basis axes. We will worry about direction later in this notebook.

Apart from just the covariance, the speed should also affect the mean of our region of influence. Someone who is running full speed should have less influence on the ground behind them, and more on the ground in front of them. We can model this by shifting the mu in the movement direction as a function of velocity.

Let's plot this out, first by taking the low hanging fruit and plot out the mean shift. I will draw an arrow to signify the velocity of the player. We will shift the mean according to the following function

$\mathbf{\mu^*} = \mathbf{\mu} + 0.5\mathbf{v}$

For a player at the position $(0, 0)$ with a velocity vector $(6, 0)$ and no changes to the covariance matrix yet will have a covariance matrix as follows

In [ ]:
# Player positions
player_throw = np.array([0, 0])

sigma_throw = np.array([[1**2, 0], [0, 1**2]])
mu_throw = player_throw
vel_throw = np.array([6, 0])

mu_throw = mu_throw + 0.5*vel_throw

rv_throw = stats.multivariate_normal(mu_throw, sigma_throw)

# Create grid of points to pass into multivariate normal
x, y = np.mgrid[-10:10:.01, -10:10:.01]
pos = np.dstack((x, y))

# Plot players and 
fig = plt.figure(figsize = (10, 10))
ax = plt.gca()
ax_influence = ax.twinx()

# Plot the figure
ax_influence.contourf(x, y, rv_throw.pdf(pos), alpha = 0.3, cmap = 'Greys')

ax.scatter(player_throw[0], player_throw[1], s = 300, color = 'k', label = 'thrower')
ax.arrow(0, 0, 6, 0, head_width = 0.2, color = 'k')

ax.set_xlim([-10, 10])
ax.set_ylim([-10, 10])

ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)
ax_influence.yaxis.set_visible(False)

ax.set

fig.show()

Now for the covariance, we need to stretch the covariance matrix along the axis that we are moving, and shrink it in the orthogonal direction as a function of velocity and distance from the ball. Since we know our covariance matrix is diagonal, this is easy as we just need to worry about the individual variances. For now, let's take the distance to the ball to be 0. The bigger the distance, the wider the covariance should be in either direction. 

The initial covariance matrix we will use will be created from our distance-from-ball function $f_d(\cdot)$.

$
\Sigma=
  \begin{bmatrix}
    f_d(t) & 0 \\
    0 & f_d(t)
  \end{bmatrix}
$

We will increase this in the direction of movement (in this case in the $x$ direction). To incorporate a weighting with speed, we will take the maximum possible speed and create a weighting function. The maximum possible recorded speed as of the time of writing this notebook is 23.1 mph set by Raheem Mostert of the 49ers. This corresponds to approximately 11.3 yards per second. We don't want this to be a linear weighting either. At lower speeds, it is still easier to change direction than at higher speeds so we want the weighting to be exponential. If a player reaches a maximum speed, we want the variance to be $f_d(t)$. If the player is at 0 speed, we want the variance to be $\frac{f_d(t)}{2}$. This seems to have been set relatively arbitrarily by the authors, but if anyone has a reason as to why they did it this way, please let me know. In either case, you can tune the hyperparameters to ensure you are getting the correct radius of influence.

To meet the requirements of the speed weighting function, we can do $f_s(s) = \frac{x^2}{11.3^2}$. Squaring will provide the effect we require. You can compare this with a linear weighting in the graph below

In [ ]:
x = np.arange(0,13,0.1)

plt.plot(x/13, 'k', label = 'linear')
plt.plot(x**2/13**2, 'r', label = 'exponential')

plt.legend()

plt.show()

The covariance can now just be written as a simple weighting

$
\Sigma=
  \begin{bmatrix}
    \frac{f_d(t) + f_d(t) \cdot f_s(s)}{2} & 0 \\
    0 & \frac{f_d(t) - f_d(t) \cdot f_s(s)}{2}
  \end{bmatrix}
$

Plotting this, we get

In [ ]:
def speed_weighting(s):
    return (s**2)/(11.3**2)

player_speed = 6

# Player positions
player_throw = np.array([0, 0])

sigma_throw = np.array([[(radius_influence(0) + radius_influence(0)*speed_weighting(player_speed))/2, 0], [0, (radius_influence(0) - radius_influence(0)*speed_weighting(player_speed))/2]])
mu_throw = player_throw
vel_throw = np.array([player_speed, 0])

mu_throw = mu_throw + 0.5*vel_throw

rv_throw = stats.multivariate_normal(mu_throw, sigma_throw)

# Create grid of points to pass into multivariate normal
x, y = np.mgrid[-10:10:.01, -10:10:.01]
pos = np.dstack((x, y))

# Plot players and 
fig = plt.figure(figsize = (10, 10))
ax = plt.gca()
ax_influence = ax.twinx()

# Plot the figure
ax_influence.contourf(x, y, rv_throw.pdf(pos), alpha = 0.3, cmap = 'Greys')

ax.scatter(player_throw[0], player_throw[1], s = 300, color = 'k', label = 'thrower')
ax.arrow(0, 0, player_speed, 0, head_width = 0.2, color = 'k')

ax.set_xlim([-10, 10])
ax.set_ylim([-10, 10])

ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)
ax_influence.yaxis.set_visible(False)

ax.set

fig.show()

Note how the covariance extends based on the speed of the player. This is exactly the behaviour we wanted.

## Modelling how direction affects region of influence
While this works if the player is moving in the x or y direction, we need to reassess how we create the covariance matrix to work for a player travelling in any direction. To do this, we can use the spectral theorem knowing that our covariance matrix is symmetric and real.

>**Spectral Theorem**:
Given $A \in n\times n$ is real, symmetric matrix, it can be factorized into $A=V S V^\top$ where the columns of $V$ contain orthonormal eigenvectors, and the diagonal of $\Sigma$ contains the corresponding eigenvalues.

The spectral theorem is used as a basies for PCA, and the eigenvectors will encode the orthogonal directions of highest variance. I will not go into the proof here, but for a very good tutorial on PCA, please see this [paper](https://arxiv.org/pdf/1404.1100.pdf) by Jonathan Shlens. Now, knowing what we know, we can work on backwards engineering a valid covariance matrix. 

First let's assume that we are taking our orientation from the x axis moving counter clockwise. By this I mean we would start at vector $(1, 0)$, and then rotate counter clockwise. Be aware that the data given by NFL starts at vector $(0, 1)$ and rotates clockwise. We will attempt to recreate $V$ first. As the columns of V will correspond to the two largest orthogonal degrees of variance, we can just think of V as a an orthonormal rotation matrix.

To better illustrate this, let us do an example. Assume that the direction is $\pi/4$ radians, or 45 degrees. We want to move our vector $(1, 0)$ to the corresponding point at 45 degrees on the unit circle. Using basic trigonometry, this is $(\cos(\theta), \sin(\theta))$. Similarly, we want to move the vector $(0, 1)$ to the corresponding point at 135 degrees. Again using basic trigonometry, this corresponds to $(-\sin(\theta), \cos(\theta))$. Both of these directions are unit normal (they are on the unit circle) and orthogonal (they are perpendicular). As such they are valid for use as orthonormal eigenvectors. As such, we can create our matrix as

$
V=
  \begin{bmatrix}
    \cos(\theta) & -\sin(\theta) \\
    \sin(\theta) & \cos(\theta)
  \end{bmatrix}
$

We will use the same eigenvalues for our matrix that we did last time. This will stretch the rotate the principle components in the corresponding direction. As a reminder, this matrix is

$
S=
  \begin{bmatrix}
    \frac{f_d(t) + f_d(t) \cdot f_s(s)}{2} & 0 \\
    0 & \frac{f_d(t) - f_d(t) \cdot f_s(s)}{2}
  \end{bmatrix}
$

Using the spectral theorem, we can now write out our new covariance matrix as

$\Sigma = VSV^\top$

To show this in an example, I will first plot the rotation of the bases, and then the rotated multivariate gaussian.

In [ ]:
fig = plt.figure(figsize = (10, 10))
ax = plt.gca()

theta = np.pi/4

ax.arrow(0,0,1,0, color = 'k', head_width = 0.05)
ax.arrow(0,0,0,1, color = 'k', head_width = 0.05)
ax.arrow(0,0,np.cos(theta),np.sin(theta), color = 'r', head_width = 0.05)
ax.arrow(0,0,-np.sin(theta),np.cos(theta), color = 'r', head_width = 0.05)

ax.set_xlim([-1.5, 1.5])
ax.set_ylim([-1.5, 1.5])

ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)

fig.show()


In [ ]:
def speed_weighting(s):
    return (s**2)/(11.3**2)

player_speed = 6
influence_rad = np.pi/4

# Player positions
player_throw = np.array([0, 0])

sigma_throw = np.array([[(radius_influence(0) + radius_influence(0)*speed_weighting(player_speed))/2, 0], [0, (radius_influence(0) - radius_influence(0)*speed_weighting(player_speed))/2]])
mu_throw = player_throw
vel_throw = np.array([np.sqrt(18), np.sqrt(18)])

R = np.array([[np.cos(influence_rad), -np.sin(influence_rad)],[np.sin(influence_rad), np.cos(influence_rad)]])

sigma_rotated = R@sigma_throw@R.T

mu_throw = mu_throw + 0.5*vel_throw

# rv_throw = stats.multivariate_normal(mu_throw, sigma_throw)
rv_throw = stats.multivariate_normal(mu_throw, sigma_rotated)

# Create grid of points to pass into multivariate normal
x, y = np.mgrid[-10:10:.01, -10:10:.01]
pos = np.dstack((x, y))

# Plot players and 
fig = plt.figure(figsize = (10, 10))
ax = plt.gca()
ax_influence = ax.twinx()

# Plot the figure
ax_influence.contourf(x, y, rv_throw.pdf(pos), alpha = 0.3, cmap = 'Reds')

ax.scatter(player_throw[0], player_throw[1], s = 300, color = 'r', label = 'thrower')
ax.arrow(0, 0, vel_throw[0], vel_throw[1], head_width = 0.2, color = 'r')

ax.set_xlim([-10, 10])
ax.set_ylim([-10, 10])

ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)
ax_influence.yaxis.set_visible(False)

ax.set

fig.show()

Note how using the above formulation, we can now easily rotate the regions of influence as we like!

That is it! That brings to a close the section of regions of player influence. We now create such a region of influence for every single player in the team, and then we can move on to the pitch control section.

In [ ]:
class AnimatePlayPitchControl(AnimatePlay):
    def __init__(self, play_df, plot_size_len, show_control = True) -> None:
        super().__init__(play_df, plot_size_len)
        """Initializes the datasets used to animate the play.

        Parameters
        ----------
        play_df : DataFrame
            Dataframe corresponding to the play information for the play that requires
            animation. This data will come from the weeks dataframe and contains position
            and velocity information for each of the players and the football.

        Returns
        -------
        None
        """
        self._MAX_PLAYER_SPEED = 11.3
        self._X, self._Y, self._pos = self.generate_data_grid()
        
        self._ax_football = self._ax_field.twinx()
        
        self._show_control = show_control
        plt.close()
    
    @staticmethod
    @np.vectorize
    def radius_influence(x):
        assert x >= 0

        if x <= 18:
            return 4 + (6/(18**2))*(x**2)
        else:
            return 10
        
    def generate_data_grid(self, N = 120):
        # Our 2-dimensional distribution will be over variables X and Y
        X = np.linspace(0, self._MAX_FIELD_X, N)
        Y = np.linspace(0, self._MAX_FIELD_Y, N)
        X, Y = np.meshgrid(X, Y)

        # # Mean vector and covariance matrix
        # mu = np.array([0., 1.])
        # Sigma = np.array([[ 1. , -0.5], [-0.5,  1.5]])

        # Pack X and Y into a single 3-dimensional array
        pos = np.empty(X.shape + (2,))
        pos[:, :, 0] = X
        pos[:, :, 1] = Y
        
        return X, Y, pos
    
    @staticmethod
    def sigmoid(x, k):
        return 1 / (1 + np.exp(-k*x))

    @staticmethod
    def weighted_angle(x1, x2, w):
        def normalize(v):
            norm=np.linalg.norm(v, ord=1)
            if norm==0:
                norm=np.finfo(v.dtype).eps
            return v/norm

        norm_weighted = w*normalize(x1) + (1-w)*normalize(x2)

        return np.arctan2(norm_weighted[1], norm_weighted[0]) % (2*np.pi)
        
    @staticmethod
    def multivariate_gaussian(pos, mu, Sigma):
        """Return the multivariate Gaussian distribution on array pos.

        pos is an array constructed by packing the meshed arrays of variables
        x_1, x_2, x_3, ..., x_k into its _last_ dimension.

        """

        n = mu.shape[0]
        Sigma_det = np.linalg.det(Sigma)
        Sigma_inv = np.linalg.inv(Sigma)
        N = np.sqrt((2*np.pi)**n * Sigma_det)
        # This einsum call calculates (x-mu)T.Sigma-1.(x-mu) in a vectorized
        # way across all the input variables.
        fac = np.einsum('...k,kl,...l->...', pos-mu, Sigma_inv, pos-mu)

        return np.exp(-fac / 2) / N
        
    def generate_sigma(self, influence_rad, player_speed, distance_from_football):
        R = np.array([[np.cos(influence_rad), -np.sin(influence_rad)],[np.sin(influence_rad), np.cos(influence_rad)]])

        speed_ratio = (player_speed**2)/(self._MAX_PLAYER_SPEED**2)

        S = np.array([[self.radius_influence(distance_from_football) + (self.radius_influence(distance_from_football)*speed_ratio), 0], 
        [0, self.radius_influence(distance_from_football) - (self.radius_influence(distance_from_football)*speed_ratio)]])
        
        return R@(S**2)@R.T
    
    def generate_mu(self, player_position, player_vel):
        return player_position + 0.5*player_vel
    
    def setup_plot(self): 
        self.set_axis_plots(self._ax_field, self._MAX_FIELD_X, self._MAX_FIELD_Y)
        
        ball_snap_df = self._frame_data[(self._frame_data.event == 'ball_snap') & (self._frame_data.team == 'football')]
        self._ax_field.axvline(ball_snap_df.x.to_numpy()[0], color = 'k', linestyle = '--')
        
        self.set_axis_plots(self._ax_home, self._MAX_FIELD_X, self._MAX_FIELD_Y)
        self.set_axis_plots(self._ax_away, self._MAX_FIELD_X, self._MAX_FIELD_Y)
        self.set_axis_plots(self._ax_jersey, self._MAX_FIELD_X, self._MAX_FIELD_Y)
        self.set_axis_plots(self._ax_football, self._MAX_FIELD_X, self._MAX_FIELD_Y)
        
        for idx in range(10,120,10):
            self._ax_field.axvline(idx, color = 'k', linestyle = '-', alpha = 0.05)
            
        self._scat_football = self._ax_football.scatter([], [], s = 100, color = 'black')
        self._scat_home = self._ax_home.scatter([], [], s = 500, color = self._CPLT[0], edgecolors = 'k')
        self._scat_away = self._ax_away.scatter([], [], s = 500, color = self._CPLT[1], edgecolors = 'k')
        
        self._scat_jersey_list = []
        self._scat_number_list = []
        self._scat_name_list = []
        self._a_dir_list = []
        self._a_or_list = []
        self._inf_contours_list = []
        for _ in range(self._MAX_FIELD_PLAYERS):
            self._scat_jersey_list.append(self._ax_jersey.text(0, 0, '', horizontalalignment = 'center', verticalalignment = 'center', c = 'white'))
            self._scat_number_list.append(self._ax_jersey.text(0, 0, '', horizontalalignment = 'center', verticalalignment = 'center', c = 'black'))
            self._scat_name_list.append(self._ax_jersey.text(0, 0, '', horizontalalignment = 'center', verticalalignment = 'center', c = 'black'))
            
            self._a_dir_list.append(self._ax_field.add_patch(Arrow(0, 0, 0, 0, color = 'k')))
            self._a_or_list.append(self._ax_field.add_patch(Arrow(0, 0, 0, 0, color = 'k')))
            
            if not self._show_control:
                self._inf_contours_list.append(self._ax_field.contourf([0, 0], [0, 0], [[0,0],[0,0]]))
        
        if self._show_control:
            self._pitch_control_contour = self._ax_field.contourf([0, 0], [0, 0], [[0,0],[0,0]])
            
        return (self._scat_football, self._scat_home, self._scat_away, *self._scat_jersey_list, *self._scat_number_list, *self._scat_name_list)
        
    def update(self, anim_frame):
        pos_df = next(self._stream)
        
        for label in pos_df.team.unique():
            label_data = pos_df[pos_df.team == label]

            if label == 'home':
                self._scat_home.set_offsets(np.vstack([label_data.x, label_data.y]).T)
            elif label == 'away':
                self._scat_away.set_offsets(np.vstack([label_data.x, label_data.y]).T)
            elif label == 'football':
                self._scat_football.set_offsets(np.hstack([label_data.x, label_data.y]))

        jersey_df = pos_df[pos_df.jerseyNumber.notnull()]
        
        inf_home_team = 0
        inf_away_team = 0
        
        for (index, row) in pos_df[pos_df.jerseyNumber.notnull()].reset_index().iterrows():
            self._scat_jersey_list[index].set_position((row.x, row.y))
            self._scat_jersey_list[index].set_text(row.position)
            self._scat_number_list[index].set_position((row.x, row.y+1.9))
            self._scat_number_list[index].set_text(int(row.jerseyNumber))
            self._scat_name_list[index].set_position((row.x, row.y-1.9))
            self._scat_name_list[index].set_text(row.displayName.split()[-1])
            
            player_orientation_rad = self.deg_to_rad(self.convert_orientation(row.o))
            player_direction_rad = self.deg_to_rad(self.convert_orientation(row.dir))
            player_speed = row.s
            player_position = np.array([row.x, row.y])
            player_acc = row.a
            
            speed_w = player_speed/self._MAX_PLAYER_SPEED
            
            player_vel = np.array([np.real(self.polar_to_z(player_speed, player_direction_rad)), np.imag(self.polar_to_z(player_speed, player_direction_rad))])
            player_orient = np.array([np.real(self.polar_to_z(2, player_orientation_rad)), np.imag(self.polar_to_z(2, player_orientation_rad))])
            
            influence_rad = self.weighted_angle(player_vel, player_orient, speed_w)
            
            distance_from_football = np.sqrt((pos_df[pos_df.displayName == 'Football'].x - player_position[0])**2 + ((pos_df[pos_df.displayName == 'Football'].y - player_position[1]))**2).to_numpy()[0]
            
            self._a_dir_list[index].remove()
            self._a_dir_list[index] = self._ax_field.add_patch(Arrow(row.x, row.y, player_vel[0], player_vel[1], color = 'k'))
            
            self._a_or_list[index].remove()
            self._a_or_list[index] = self._ax_field.add_patch(Arrow(row.x, row.y, player_orient[0], player_orient[1], color = 'grey', width = 2))
            
            sigma = self.generate_sigma(influence_rad, player_speed, distance_from_football)
            mu = self.generate_mu(player_position, player_vel)
            
            Z = self.multivariate_gaussian(self._pos, mu, sigma)
            Z_coarse = np.where(Z > 0.001, Z, np.nan)
            
            if not self._show_control:
                for cont_info in self._inf_contours_list[index].collections:
                    cont_info.remove()
            
            if row.team == 'home':
                if self._show_control:
                    inf_home_team += Z
                else:
                    self._inf_contours_list[index] = self._ax_field.contourf(self._X, self._Y, Z_coarse, cmap='Reds', levels = 10, alpha = 0.1)
            elif row.team == 'away':
                if self._show_control:
                    inf_away_team += Z
                else:
                    self._inf_contours_list[index] = self._ax_field.contourf(self._X, self._Y, Z_coarse, cmap='Greens', levels = 10, alpha = 0.1)
        
        if self._show_control:
            for cont_info in self._pitch_control_contour.collections:
                    cont_info.remove()
                
            self._pitch_control_contour = self._ax_field.contourf(self._X, self._Y, self.sigmoid(inf_away_team/len(pos_df[pos_df.team=='away']) - inf_home_team/len(pos_df[pos_df.team=='home']),k = 1000), levels = 50, cmap='PiYG', vmin = 0.45, vmax = 0.55, alpha = 0.7)
            
#             self._fig.colorbar(self._pitch_control_contour, extend='min', shrink=0.9, ax=self._ax_field)
        
        return (self._scat_football, self._scat_home, self._scat_away, *self._scat_jersey_list, *self._scat_number_list, *self._scat_name_list)

In [ ]:
animated_play = AnimatePlayPitchControl(play_df, 20, show_control=False)
HTML(animated_play.ani.to_jshtml())

In [ ]:
animated_play = AnimatePlayPitchControl(play_df, 20, show_control=True)
HTML(animated_play.ani.to_jshtml())